# Notebook Setup

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.model_selection import train_test_split as tts
import datetime

# Dela - Predicting the amount of deaths per year

### 1. Intro
<b>Who is our client?</b><br>
Our client is Dela. Dela is a funeral insurer and caretaker for funeral services. In this semester, they will give us some inside problems that we can investigate on.<br><br>
<b>Project explanation</b><br>
Dela was faced with unprecedented challenge by the fluctuating demands during the Covid19 first year, and that way they are looking to enhance their abilities to react to surprising serge or drops in demands.
We cannot predict when Dela needs to upscale or downscale. However, we can forecast the amount of deaths in the upcoming years. Based on that knowledge and Dela experience, they can conclude themselves when to upscale or downscale.<br><br>
<b>Project goal</b><br>
In our project, we are going to forecast the amount of deaths per year. In this way we can help them conclude easier what to do on higher, lower demand.<br><br><br>
<b>Document explanation</b><br>
In this document there will be the testing and implementation of the project delivered proposal hypothesis. In this document, we will do that by understanding and experimenting with the collected data. After the understanding, we will see if our hypothesis can be validated and we will do that by applying machine learning onto our dataset.<br><br>
<b>Document setup:</b><br>
<table style="font-size: 14px !important; margin: 0 !important">
    <tr>
        <th style="text-align: left;">Data requirements</th>
        <td style="text-align: left;">In this chapter, we are going to setup the requirements for the data that is needed for the prediction. We will answer questions like ‘Which references are trustworthy?’, ‘Do we need some specific features?’, etc…</td>
    </tr>
    <tr>
        <th style="text-align: left;">Data collection</th>
        <td style="text-align: left;">In this chapter, we are going to explain where we found our data, where we are going to store our data and references to he subchapters of each dataset</td>
    </tr>
    <tr>
        <th style="text-align: left;">Data understanding</th>
        <td style="text-align: left;">In this chapter, we are going to understand each specific dataset that we downloaded to really understand the value of each dataset and how it is going to bring a value to Dela.</td>
    </tr>
    <tr>
        <th style="text-align: left;">Data preparation</th>
        <td style="text-align: left;">In this chapter, we are going to prepare our data so it is clean to work with, think about removing all the data that has invalid records, data that has wrong values or data that has similar features with different names.</td>
    </tr>
</table>

### 2. Provisioning

### 2.1 Data Requirements
In this chapter, we are trying to setup the expectations/ requirements of the data we are going to collect for the provisioning fase.

<table style="font-size: 14px !important; margin: 0 !important">
    <tr>
        <th style="text-align: left !important">Data Domain</th>
        <td style="text-align: left !important"></td>
    </tr>
    <tr>
        <th style="text-align: left !important">Data type</th>
        <td style="text-align: left !important"></td>
    </tr>
    <tr>
        <th style="text-align: left !important">Target Variable</th>
        <td style="text-align: left !important"></td>
    </tr>
    <tr>
        <th style="text-align: left !important">Expected Features</th>
        <td style="text-align: left !important"></td>
    </tr>
</table>

### 2.2 Data Collection
Because we want to search for data that contains the amount of deaths in the Netherlands, we began searching for an open data bank that is governmental. In this case the data is from a trustworthy source and will higher the chance of a good prediction. That's when we landed on CBS (stands for `Centraal Bureau voor de Statistiek`, translated to English that means  `Central Station of Statistics`). 

<table style="font-size: 14px !important; margin: 0 !important">
    <tr>
        <th style="text-align: left !important">Data Source</th>
        <td style="text-align: left !important">We got our data from the official   <a href="https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS" target="_blank">CBS</a> Website</td>
    </tr>
    <tr>
        <th style="text-align: left !important">Data Storage</th>
        <td style="text-align: left !important">We stored all of our datasets on <a href="https://github.com/i454038/AI-car-price-prediction" target="_blank">Github</a>, so it is globally accessible</td>
    </tr>
</table>

Load in the datasets from Github

In [2]:
# these are custom classes made to keep the notebook neat.
from classes.dataImporting import datasetManager

datasets = datasetManager.defineDatasets()
dataframes = datasetManager.loadDatasets(datasets)

### 2.3 Data Understanding

We want to plot the deceased

### 2.4 Data Preperation

### LifeExpectency - LifeExpectencyPerRegion

For the life expectency
- We renamed the columns to English.
- We converted the integer columns to floats instead of objects.

In [3]:
dataframes['lifeExpectency']['lifeExpectencyPerRegion'] = (
    dataframes['lifeExpectency']['lifeExpectencyPerRegion']
        .rename(columns=datasetManager.renameFeatures('lifeExpectency.lifeExpectencyPerRegion'))
        .assign(LifeExpectancy = lambda x: x.LifeExpectancy.str.replace(',', '.').astype(float))
        .assign(LifeExpectancyWhen65OrOlder = lambda x: x.LifeExpectancyWhen65OrOlder.str.replace(',', '.').astype(float))
)

In [4]:
dataframes['lifeExpectency']['lifeExpectencyPerRegion'].head()

,id,Municipality,Groep_rij,Geslacht,LifeExpectancy,LifeExpectancyNL,LifeExpectancyWhen65OrOlder,LifeExpectancyWhen65OrOlderNL
0,518,'s-Gravenhage,Levensverwachting,Totaal,80.8,"onder, 99% zeker",19.1,"onder, 99% zeker"
1,796,'s-Hertogenbosch,Levensverwachting,Totaal,81.3,"onder, 99% zeker",19.7,"onder, 99% zeker"
2,1680,Aa en Hunze,Levensverwachting,Totaal,82.1,geen,20.4,geen
3,358,Aalsmeer,Levensverwachting,Totaal,82.9,"boven, 99% zeker",20.1,geen
4,197,Aalten,Levensverwachting,Totaal,82.1,geen,20.3,geen


### PopulationChange - Pop2002_2020

In [5]:
# transform dataset to align with others
dataframes['populationChange']['pop2002_2020'] = (
    dataframes['populationChange']['pop2002_2020']
        .rename(columns=datasetManager.renameFeatures('populationChange.pop2002_2020'))
        .assign(Year = lambda x: pd.to_datetime(x.Year.str[:4]).dt.year)
)

In [6]:
dataframes['populationChange']['pop2002_2020'].head(5)

,ID,Municipality,Year,PopulationAtBeginOfPeriod,AliveBornChildren,Deceased,TotalLocations,LocationsFromOtherMunicipality,Immigration,TotaalVertrekInclAdmCorrecties_7,AmountMovedToOtherMunicipality,EmigratieInclusiefAdmCorrecties_9,OverigeCorrecties_10,PopulationGrowth,RelativePopulationGrowth,PopulationGrowthSinceJanuari,RelativePopulationGrowthSinceJanuari,PopulationAtEndOfPeriod
0,14604,GM1680,2002,25552.0,289.0,251.0,1353.0,1121.0,232.0,1617.0,1498.0,119.0,-21.0,-247.0,-0.97,-247.0,-0.97,25305.0
1,14617,GM1680,2003,25305.0,279.0,241.0,1127.0,1071.0,56.0,1264.0,1111.0,153.0,12.0,-87.0,-0.34,-87.0,-0.34,25218.0
2,14630,GM1680,2004,25218.0,233.0,221.0,1167.0,1104.0,63.0,1077.0,1023.0,54.0,9.0,111.0,0.44,111.0,0.44,25329.0
3,14643,GM1680,2005,25329.0,230.0,231.0,1322.0,1254.0,68.0,1143.0,1062.0,81.0,0.0,178.0,0.70,178.0,0.70,25507.0
4,14656,GM1680,2006,25507.0,216.0,212.0,1369.0,1320.0,49.0,1326.0,1222.0,104.0,9.0,56.0,0.22,56.0,0.22,25563.0


### Death - Reason of Death

In [7]:
dataframes['death']['reason_per_year_per_region2002_2020'] = (
    pd.concat([
        dataframes['death']['reasons_per_year_per_region2002_2015'], 
        dataframes['death']['reasons_per_year_per_region2016_2020']
    ])
)

dataframes['death']['reason_per_year_per_region2002_2020'] = (
    dataframes['death']['reason_per_year_per_region2002_2020']
        .rename(columns={'RegioS': 'Municipality', 'Perioden': 'Year'})
        .assign(Year = lambda x: pd.to_datetime(x.Year.str[:4]).dt.year)
)

### Merge Datasets

<table style="margin: 0 !important; font-size: 14px !important">
    <tr>
        <th style="text-align: left !important">Goal:</th>
        <td style="text-align: left !important">
            Merge the datasets:<br>
            <ul>
                <li>lifeExpectency</li>
                <li>population</li>
                <li>reason of death</li>
            </ul>
        </td>
    </tr>
    <tr>
        <th style="text-align: left !important">Possible Solutions:</th>
        <td style="text-align: left !important">
            <ul>
                <li>merge (pandas)</li>
                <li>concatinate (pandas)</li>
            </ul>
        </td>
    </tr>
</table>

<b>Merge</b><br>
Merge DataFrame or named Series objects with a database-style join.<br>
A named Series object is treated as a DataFrame with a single named column.<br>
The join is done on columns or indexes. If joining columns on columns, the DataFrame indexes will be ignored. Otherwise if joining indexes on indexes or indexes on a column or columns, the index will be passed on. When performing a cross merge, no column specifications to merge on are allowed.<br>
<b>Concatinate</b><br>
Concatenate pandas objects along a particular axis with optional set logic along the other axes.<br>
Can also add a layer of hierarchical indexing on the concatenation axis, which may be useful if the labels are the same (or overlapping) on the passed axis number.<br>
<b>What is the best solution?</b><br>
Our datasets do not have the same columns, it's even so that they have diffrent columns. We want to add features to one dataset and we do not want to align the dataset on a particular axis. This is why we are using the merge function to make one big dataset.

In [8]:
dataset = (
    dataframes['populationChange']['pop2002_2020']
        .merge(dataframes['lifeExpectency']['lifeExpectencyPerRegion'], how='outer', on="Municipality")
#         .merge(dataframes['death']['reason_per_year_per_region2002_2020'], how='outer', on=['Year', "Municipality"])
        .assign(Municipality = lambda x: x.Municipality.replace(datasetManager.mapFeature('Municipality')))
        .fillna(0)
        .assign(Year = lambda x: x.Year.replace(0, 2002))
        .assign(Year = lambda x: x.Year.astype(int))
)
dataset['Year'].unique()

array([2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [9]:
# PredictingDataset = dataset.assign(Municipality = lambda x: x.Municipality.astype("category").cat.codes)
# # remove values with type string (object) because they are useless eather way
dataset = dataset[[
    i for i in list(dataset.columns) 
    if i not in 
    ['Groep_rij', 'Geslacht', 'LifeExpectancyNL', 'LifeExpectancyWhen65OrOlderNL', 'Geslacht_y']
]]

In [10]:
dataset.set_index(['Municipality','Year'], inplace=True)

In [11]:
municipalities = dataset.reset_index()['Municipality'].unique()
datasets = {}
for munucipality in municipalities:
    if(dataset.loc[munucipality].shape[0] <= 20 and dataset.loc[munucipality].shape[0] > 1):
#         print(munucipality, dataset.loc[munucipality].shape[0])
        datasets[munucipality] = dataset.loc[munucipality].drop(['ID','id'],axis=1)

# print(len(datasets.keys()))
# print(datasets['Amsterdam']['Deceased'])
# print(datasets['Zwolle']['Deceased'])
# pd.DataFrame({
#     "recall": {
#         "amsterdam": '0.82',
#         'zwolle': '0.42'
#     },
#     'accuracy': {
#         "amsterdam": '0.82',
#         'zwolle': '0.42'
#     }
# })

<table style="margin: 0 !important;">
    <tr>
        <th style="text=align: center !important">Goal:</th>
        <td style="text=align: center !important">
            Grouping the dataset from municapality to year:<br>
        </td>
    </tr>
    <tr>
        <th style="text=align: center !important">Possible Solutions:</th>
        <td style="text=align: center !important">
            <ul>
                <li>groupby</li>
                <li>sort</li>
            </ul>
        </td>
    </tr>
</table>

In [12]:
datasets['Zwolle'].head()

,PopulationAtBeginOfPeriod,AliveBornChildren,Deceased,TotalLocations,LocationsFromOtherMunicipality,Immigration,TotaalVertrekInclAdmCorrecties_7,AmountMovedToOtherMunicipality,EmigratieInclusiefAdmCorrecties_9,OverigeCorrecties_10,PopulationGrowth,RelativePopulationGrowth,PopulationGrowthSinceJanuari,RelativePopulationGrowthSinceJanuari,PopulationAtEndOfPeriod,LifeExpectancy,LifeExpectancyWhen65OrOlder
Year,,,,,,,,,,,,,,,,,
2002,109000.0,1533.0,953.0,5370.0,4804.0,566.0,4984.0,4587.0,397.0,-11.0,955.0,0.88,955.0,0.88,109955.0,0.0,0.0
2003,109955.0,1610.0,939.0,5325.0,4795.0,530.0,5204.0,4605.0,599.0,133.0,925.0,0.84,925.0,0.84,110880.0,0.0,0.0
2004,110880.0,1551.0,930.0,5338.0,4864.0,474.0,4931.0,4510.0,421.0,-8.0,1020.0,0.92,1020.0,0.92,111900.0,0.0,0.0
2005,111900.0,1657.0,941.0,5745.0,5264.0,481.0,5355.0,4729.0,626.0,72.0,1178.0,1.05,1178.0,1.05,113078.0,0.0,0.0
2006,113078.0,1650.0,942.0,6245.0,5815.0,430.0,5452.0,4809.0,643.0,56.0,1557.0,1.38,1557.0,1.38,114635.0,0.0,0.0


<b>GroupBy</b><br>
explenation<br>
<b>Sort</b><br>
explenation

### Rename columns

def doFeatureSelection(dataset, y):
    X = dataset.loc[:, dataset.columns != 'Deceased']
    sel = SelectKBest(mutual_info_regression, k=5)
    sel.fit_transform(X,y)
    cols = sel.get_support()
    features_df_new = X.iloc[:,cols].keys()
    return features_df_new

predictionArray = []
def predictByNearestNeighbors(dataset, y):    
    scaler = StandardScaler()
    X_new = scaler.fit_transform(dataset)
    
    from sklearn.neighbors import KNeighborsRegressor

    X_train, X_test, y_train, y_test = tts(X_new, y, test_size=0.4, random_state=20)
    
    neigh = KNeighborsRegressor(n_neighbors=5)
    neigh.fit(X_train, y_train)
    prediction = neigh.predict(X_test)
    
    predictionArray.append(prediction)
    return neigh.score(X_test, y_test)

metricsDictionary = {
    "Score": {}
}
for municipality in datasets.keys():
    y = datasets[municipality].Deceased
    dataset = datasets[municipality]
    features = doFeatureSelection(dataset, y)
    dataset = dataset[features]
    score = predictByNearestNeighbors(dataset, y)
    b=score
    metricsDictionary["Score"][municipality] = score
    
scores = pd.DataFrame(metricsDictionary, columns=['Municipality'])

scores = pd.DataFrame.from_dict(metricsDictionary).reset_index().rename(columns={"index": 'Municipality'})

print(scores['Score'].mean())

In [13]:
import itertools
import statsmodels
import statsmodels.api as sm
import matplotlib


In [14]:
pd.to_datetime(datasets['Zwolle'].index)

DatetimeIndex(['1970-01-01 00:00:00.000002002',
               '1970-01-01 00:00:00.000002003',
               '1970-01-01 00:00:00.000002004',
               '1970-01-01 00:00:00.000002005',
               '1970-01-01 00:00:00.000002006',
               '1970-01-01 00:00:00.000002007',
               '1970-01-01 00:00:00.000002008',
               '1970-01-01 00:00:00.000002009',
               '1970-01-01 00:00:00.000002010',
               '1970-01-01 00:00:00.000002011',
               '1970-01-01 00:00:00.000002012',
               '1970-01-01 00:00:00.000002013',
               '1970-01-01 00:00:00.000002014',
               '1970-01-01 00:00:00.000002015',
               '1970-01-01 00:00:00.000002016',
               '1970-01-01 00:00:00.000002017',
               '1970-01-01 00:00:00.000002018',
               '1970-01-01 00:00:00.000002019',
               '1970-01-01 00:00:00.000002020',
               '1970-01-01 00:00:00.000002002'],
              dtype='datetime64[ns]', n

In [15]:
datasets['Zwolle'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 2002 to 2002
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   PopulationAtBeginOfPeriod             20 non-null     float64
 1   AliveBornChildren                     20 non-null     float64
 2   Deceased                              20 non-null     float64
 3   TotalLocations                        20 non-null     float64
 4   LocationsFromOtherMunicipality        20 non-null     float64
 5   Immigration                           20 non-null     float64
 6   TotaalVertrekInclAdmCorrecties_7      20 non-null     float64
 7   AmountMovedToOtherMunicipality        20 non-null     float64
 8   EmigratieInclusiefAdmCorrecties_9     20 non-null     float64
 9   OverigeCorrecties_10                  20 non-null     float64
 10  PopulationGrowth                      20 non-null     float64
 11  RelativePopulati

In [16]:

import itertools
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Examples of parameter combinations for Seasonal ARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 12)
SARIMAX: (0, 0, 1) x (0, 1, 0, 12)
SARIMAX: (0, 1, 0) x (0, 1, 1, 12)
SARIMAX: (0, 1, 0) x (1, 0, 0, 12)


In [17]:
import warnings
warnings.filterwarnings("ignore") # specify to ignore warning messages

arrayy=[]

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(datasets['Zwolle']['Deceased'],
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()
            arrayy.append(results.aic)
            print('SARIMAX{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

SARIMAX(0, 0, 0)x(0, 0, 0, 12)12 - AIC:314.8434495471905
SARIMAX(0, 0, 0)x(0, 0, 1, 12)12 - AIC:115.33427282244723
SARIMAX(0, 0, 0)x(0, 1, 0, 12)12 - AIC:103.82874759593327
SARIMAX(0, 0, 0)x(0, 1, 1, 12)12 - AIC:4.0
SARIMAX(0, 0, 0)x(1, 0, 0, 12)12 - AIC:118.92194504987319
SARIMAX(0, 0, 0)x(1, 0, 1, 12)12 - AIC:107.46549528518904
SARIMAX(0, 0, 0)x(1, 1, 0, 12)12 - AIC:4.0
SARIMAX(0, 0, 0)x(1, 1, 1, 12)12 - AIC:6.0
SARIMAX(0, 0, 1)x(0, 0, 0, 12)12 - AIC:279.8526199624245
SARIMAX(0, 0, 1)x(0, 0, 1, 12)12 - AIC:97.27315605154965
SARIMAX(0, 0, 1)x(0, 1, 0, 12)12 - AIC:89.73145421390464
SARIMAX(0, 0, 1)x(0, 1, 1, 12)12 - AIC:6.0
SARIMAX(0, 0, 1)x(1, 0, 0, 12)12 - AIC:118.62988667077974
SARIMAX(0, 0, 1)x(1, 0, 1, 12)12 - AIC:94.161264993309
SARIMAX(0, 0, 1)x(1, 1, 0, 12)12 - AIC:6.0
SARIMAX(0, 0, 1)x(1, 1, 1, 12)12 - AIC:8.0
SARIMAX(0, 1, 0)x(0, 0, 0, 12)12 - AIC:251.51101687013295
SARIMAX(0, 1, 0)x(0, 0, 1, 12)12 - AIC:93.65786377473162
SARIMAX(0, 1, 0)x(0, 1, 0, 12)12 - AIC:91.977461305296

In [18]:
mod = sm.tsa.statespace.SARIMAX(datasets['Zwolle']['Deceased'],
                                order=(0, 1, 0),
                                seasonal_order=(1, 1, 0, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])

                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.S.L12            0         -0        nan        nan           0           0
sigma2      1.481e+05         -0       -inf      0.000    1.48e+05    1.48e+05


In [19]:
results.plot_diagnostics(variable=0,figsize=(15, 12))
plt.show()

ValueError: Length of endogenous variable must be larger the the number of lags used in the model and the number of observations burned in the log-likelihood calculation.

<Figure size 1080x864 with 0 Axes>

In [ ]:
pred = results.get_prediction(dynamic=False,shuffle=True)
pred_ci = pred.conf_int()

In [ ]:
ax = datasets['Zwolle']['Deceased'][2003:].plot(label='observed', figsize=(20, 15))
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('date')
ax.set_ylabel('deceased')
plt.legend()

plt.show()

In [ ]:
dd

In [ ]:
co2_forecasted = pred.predicted_mean
pd.to_datetime(datasets['Zwolle'].index)
co2_truth = datasets['Zwolle']['Deceased'][2003:]

# Compute the mean square error
mse = ((co2_forecasted - co2_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

In [ ]:
pred_dynamic = results.get_prediction(start=2003 ,dynamic=True, full_results=True)
pred_dynamic_ci = pred_dynamic.conf_int()

In [ ]:
pd.to_datetime(datasets['Zwolle'].index)
ax = datasets['Zwolle']['Deceased'][2003:20018].plot(label='observed', figsize=(20, 15))
pred_dynamic.predicted_mean.plot(label='Dynamic Forecast', ax=ax)

ax.fill_between(pred_dynamic_ci.index,
                pred_dynamic_ci.iloc[:, 0],
                pred_dynamic_ci.iloc[:, 1], color='k', alpha=.25)

ax.fill_betweenx(ax.get_ylim(), 2003, dd.index[-1],
                 alpha=.1, zorder=-1)

ax.set_xlabel('Date')
ax.set_ylabel('CO2 Levels [ppmv]')

plt.legend()
plt.show()

# sns.scatterplot(data=scores, x='Municipality', y='Score', hue="Municipality")
import plotly.express as px
fig = px.scatter(scores, x='Municipality', y='Score', color='Municipality')
fig.show()

In [ ]:
# dataset = (
#     dataset
#         .groupby(['Year', 'Municipality']).mean()
#         .reset_index()
# )

# dataset
import plotly.express as px
px.line(dataset, x="Year", y='Deceased', color="Municipality", title="Deceased per Year per Municipality in the Netherlands")

### PopulationChange - PopComparison2015_2020

### PopulationChange - GrowthPrediction2020_2050

### Death Datasets

### Birth Datasets

<b>Clean Data</b>

- lifeExpectency
    - lifeExpectencyPerRegion2016_2019 
- populationChange
    - pop2002_2020
    - popOverview
    - popComparison2015_2020
    - growthPrediction2020_2050
    - absoluteNr
- death
    - reasons1997_2014
    - reasons2005_2012
    - reasons2013_2020
    - perWeek2020_2021
- birth
    - birthPerYear1899_2018
    - avaragesOfMonth

<b>Merge</b>

In [ ]:
# these are custom classes made to keep the notebook neat.



<b>Heatmap</b>